# Sensitivity Analysis ML


- 核心目的
进行超参数敏感性分析并生成多个dataframe。记录在不同超参数组合下的：
- 过拟合score
- Accuracy
- Precision
- Recall
- F1
- 最终预测输出的mean probability

- 业务逻辑
1. 设计正交表敏感性分析实验（L18(3^6)正交表，将729次实验减少到18次）
2. 使用模块化的训练函数（已存储在function目录下）
3. 针对不同参数组合执行敏感性分析
4. 输出目录：`Supplymentary/ML_sensitivity`

- 超参数配置
- **基础配置**：learning_rate=0.001, resnet_layers=[128,128,64], d_model=64, num_heads=4, num_layers=2, dropout_rate=0.3
- **敏感性测试参数**：每个参数3个水平，共6个参数
- **不进行敏感性测试**：batch_size, epochs



In [1]:
# ========== 环境配置 ==========
import warnings
import os
import sys

# 1. 设置环境变量
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# 2. 直接重定向stderr（临时）
_old_stderr = sys.stderr
sys.stderr = open(os.devnull, 'w')

# 3. 导入matplotlib和tensorflow
try:
    import matplotlib
    import matplotlib.font_manager as fm
    matplotlib.rcParams['font.family'] = 'DejaVu Sans'
    
    import tensorflow as tf
    tf.get_logger().setLevel('ERROR')
finally:
    # 4. 恢复stderr
    sys.stderr.close()
    sys.stderr = _old_stderr

# 5. 设置警告过滤
import sklearn
warnings.filterwarnings('ignore', category=sklearn.base.InconsistentVersionWarning)
warnings.filterwarnings('ignore', module='matplotlib')


import os
import time
import tensorflow as tf

print("="*80)
print("GPU环境配置检查 - WSL Ubuntu + bayes-gpu")
print("="*80)

# ---------------------------
# 0. 基本信息（可选）
# ---------------------------
print(f"TensorFlow 版本: {tf.__version__}")
# 尝试打印构建时 CUDA/cuDNN 版本（不同版本字段名可能不同）
try:
    from tensorflow.sysconfig import get_build_info
    bi = get_build_info()
    print("CUDA build:", bi.get("cuda_version"))
    print("cuDNN build:", bi.get("cudnn_version"))
except Exception:
    pass

# ---------------------------
# 1. 检查并配置 GPU
# ---------------------------
physical_gpus = tf.config.list_physical_devices("GPU")
if physical_gpus:
    print(f"\n✅ 检测到 {len(physical_gpus)} 个 GPU:")
    # 先配置显存策略（必须在任何 GPU 初始化之前）
    try:
        for g in physical_gpus:
            tf.config.experimental.set_memory_growth(g, True)
        print("✅ GPU 显存动态增长已启用")
    except RuntimeError as e:
        print(f"⚠️ 显存动态增长设置失败（可能已初始化）：{e}")

    # 打印设备细节
    for i, g in enumerate(physical_gpus):
        print(f"   GPU {i}: {g.name}")
        try:
            det = tf.config.experimental.get_device_details(g)
            # 常见键：'device_name', 'compute_capability'
            print(f"       设备详情: {det}")
        except Exception:
            pass
else:
    print("\n⚠️ 未检测到 GPU，将使用 CPU")

# ---------------------------
# 2. 小测试：GPU 计算与计时
# ---------------------------
print("\n" + "="*80)
print("GPU性能测试")
print("="*80)

use_gpu = len(physical_gpus) > 0
device_str = "/GPU:0" if use_gpu else "/CPU:0"

# 暖机（避免第一次调用的加载开销影响计时）
with tf.device(device_str):
    _ = tf.matmul(tf.random.normal([256, 256]), tf.random.normal([256, 256]))  # warm-up
    _ = tf.matmul(tf.random.normal([256, 256]), tf.random.normal([256, 256]))  # warm-up

# 计时测试（确保同步）
size = 2000  # 如需更明显差异可调大到 4096
with tf.device(device_str):
    a = tf.random.normal([size, size])
    b = tf.random.normal([size, size])
    t0 = time.time()
    c = tf.matmul(a, b)
    _ = c.numpy()  # 同步到主机，确保计时真实
    elapsed_ms = (time.time() - t0) * 1000

print(f"✅ 设备: {device_str}")
print(f"✅ 矩阵乘法 {size}x{size} 耗时: {elapsed_ms:.2f} ms")
if use_gpu:
    print("🚀 GPU 加速正常（与 CPU 相比应明显更快）")
print("="*80)


GPU环境配置检查 - WSL Ubuntu + bayes-gpu
TensorFlow 版本: 2.19.0
CUDA build: None
cuDNN build: None

⚠️ 未检测到 GPU，将使用 CPU

GPU性能测试
✅ 设备: /CPU:0
✅ 矩阵乘法 2000x2000 耗时: 39.61 ms


## 1、Load data

In [1]:
import sys
import os
from pathlib import Path

# 最可靠的方法：查找包含data和function目录的项目根目录
def find_project_root(start_path=None):
    """查找项目根目录（包含data和function目录的目录）"""
    if start_path is None:
        start_path = Path.cwd()
    
    current = Path(start_path).resolve()
    
    # 向上查找，直到找到包含data和function目录的目录
    for _ in range(5):  # 最多向上查找5层
        if (current / 'data').exists() and (current / 'function').exists():
            return current
        parent = current.parent
        if parent == current:  # 到达根目录
            break
        current = parent
    
    # 如果找不到，假设当前目录的父目录是项目根目录
    return Path.cwd().parent

project_root = find_project_root()

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

DATA_PATH = project_root / 'data'

print(f"项目根目录: {project_root}")
print(f"数据路径: {DATA_PATH}")


项目根目录: C:\Dev\Landuse_Zhong_clean
数据路径: C:\Dev\Landuse_Zhong_clean\data


In [2]:
%load_ext autoreload
%autoreload 2 
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import json
import geopandas as gpd
from shapely.geometry import Point
from function import (
    load_embedding, load_abandon, fill_nonpositive_with_nearest, 
    filter_duplicates, NUMERIC_FEATURES, CAT_COLS, PATHS
)
from function.pipeline import run_correct_training_pipeline
from function.model_saving import save_complete_model_pipeline
from datetime import datetime
import platform
from tqdm.auto import tqdm

def normalize_path(path):
    """
    跨平台路径规范化
    将Windows路径中的反斜杠转换为正斜杠（在Linux/WSL中）
    Windows: r'data\file.shp' -> r'data\file.shp' (保持不变)
    Linux/WSL: r'data\file.shp' -> 'data/file.shp' (转换)
    """
    if isinstance(path, str):
        # 检测是否在WSL或Linux环境中
        is_linux = platform.system() in ['Linux', 'Darwin']
        
        if is_linux:
            # 在Linux/WSL中，将反斜杠转为正斜杠
            path = path.replace('\\', '/')
        
        return path
    return path

def normalize_paths_dict(paths_dict):
    """
    规范化路径字典中的所有路径
    """
    normalized = {}
    for key, value in paths_dict.items():
        normalized[key] = normalize_path(value)
    return normalized

def clip_data_with_us_states(df, us_states_gdf, lon_col='lon', lat_col='lat'):
    """
    使用美国州界 shapefile 剪裁点数据；兼容不同版本的 geopandas 参数名
    """
    geometry = [Point(xy) for xy in zip(df[lon_col], df[lat_col])]
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs='EPSG:4326')
    us_states_4326 = us_states_gdf.to_crs('EPSG:4326')

    try:
        clipped = gpd.sjoin(gdf, us_states_4326, how='inner', predicate='within')
    except TypeError:
        # 旧版本 geopandas 使用 op 参数
        clipped = gpd.sjoin(gdf, us_states_4326, how='inner', op='within')

    # 清理 shapefile 附加字段
    clipped = clipped.drop(columns=['geometry', 'index_right'], errors='ignore')
    for col in us_states_gdf.columns:
        if col in clipped.columns:
            clipped = clipped.drop(columns=[col], errors='ignore')
    return clipped

PATHS = normalize_paths_dict(PATHS)
PATHS['us_pv_embedding'] = normalize_path(str(DATA_PATH / 'training_embedding.csv'))
PATHS['us_abandon'] = normalize_path(str(DATA_PATH / 'us_abandon_clean.csv'))
print(f"us_abandon path: {PATHS['us_abandon']}")


usa_bounds_main = dict(lon_min=-125, lon_max=-65, lat_min=25, lat_max=49)
# 加载数据
print("加载数据...")
df_embedding = load_embedding(PATHS['us_pv_embedding'])
df_abandon = load_abandon(PATHS['us_abandon'])

df_abandon = load_abandon(PATHS['us_abandon'])
df_embedding = load_embedding(PATHS['us_pv_embedding'])

# 初步经纬度范围过滤
df_embedding = df_embedding[
    (df_embedding['lon'] >= usa_bounds_main['lon_min']) &
    (df_embedding['lon'] <= usa_bounds_main['lon_max']) &
    (df_embedding['lat'] >= usa_bounds_main['lat_min']) &
    (df_embedding['lat'] <= usa_bounds_main['lat_max'])
]

df_abandon = df_abandon[
    (df_abandon['lon'] >= usa_bounds_main['lon_min']) &
    (df_abandon['lon'] <= usa_bounds_main['lon_max']) &
    (df_abandon['lat'] >= usa_bounds_main['lat_min']) &
    (df_abandon['lat'] <= usa_bounds_main['lat_max'])
]
# 第二次用州界矢量更精确裁剪
us_nation = gpd.read_file(normalize_path(str(DATA_PATH / 'US_data' / 'cb_2018_us_nation_5m.shp')))
df_abandon = clip_data_with_us_states(df_abandon, us_nation)
df_embedding = clip_data_with_us_states(df_embedding, us_nation)

# 数据预处理
print("预处理数据...")
df_embedding_fill = fill_nonpositive_with_nearest(df_embedding)
df_abandon_fill = fill_nonpositive_with_nearest(df_abandon)
df_abandon_filtered = filter_duplicates(df_abandon_fill, df_embedding_fill)

# 定义特征列表（排除坐标）
features_no_coords = [f for f in (NUMERIC_FEATURES + CAT_COLS) if f not in ['lat', 'lon']]
features_no_coords = [c for c in features_no_coords if c in df_embedding_fill.columns]

print(f"✅ 数据加载完成")
print(f"  - df_embedding_fill: {len(df_embedding_fill)} 行")
print(f"  - df_abandon_filtered: {len(df_abandon_filtered)} 行")
print(f"  - features_no_coords: {len(features_no_coords)} 个特征")


gogogo


I0000 00:00:1764019411.733590  147604 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
I0000 00:00:1764019411.878892  147604 cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI AVX_VNNI_INT8 AVX_NE_CONVERT FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
I0000 00:00:1764019413.225357  147604 port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


[OK] TensorFlow available
[OK] scikeras available
[OK] SHAP available
us_abandon path: /mnt/c/Dev/Landuse_Zhong_clean/data/us_abandon_clean.csv
加载数据...
You want to predict the year: 2020.0
You want to predict the year: 2020.0
预处理数据...
列 GDPpc 没有需要填充的非正值或NaN值
列 GDPpc 没有需要填充的非正值或NaN值
✅ 数据加载完成
  - df_embedding_fill: 10473 行
  - df_abandon_filtered: 70337 行
  - features_no_coords: 15 个特征


## 2、Hyperparameter settings


In [4]:
# 基础配置（base model = 各因子的中等水平）
base_configs = {
    "learning_rate": 0.001,           # 中等
    "dropout_rate": 0.3,              # 中等
    "transformer_config": {
        "d_model": 64,                # 中等宽度
        "num_heads": 4,
        "num_layers": 8,              # ← 中等深度（浅/中/深：4 / 8 / 12）
    },
    "resnet_width": 128,
    "resnet_depth": 6,
}

# 敏感性测试参数（6个因子，每个3个水平）
sensitivity_configs = {
    # 1. Transformer宽度（d_model）
    "d_model": [32, 64, 128],             # 小 / 中 / 大

    # 2. Transformer深度（num_layers）
    "num_layers": [4, 8, 12],             # 浅 / 中 / 深（差异明显）

    # 3. ResNet宽度（第一层宽度）
    "resnet_width": [64, 128, 256],       # 小 / 中 / 大

    # 4. ResNet深度（层数）
    "resnet_depth": [3, 6, 9],            # 浅 / 中 / 深（明显区分）

    # 5. 学习率
    "learning_rate": [0.0001, 0.001, 0.01],  # 小 / 中 / 大

    # 6. Dropout率
    "dropout_rate": [0.1, 0.3, 0.5],      # 弱 / 中 / 强正则
}

# 辅助函数：将resnet_width和resnet_depth转换为resnet_layers列表
def generate_resnet_layers(width, depth):
    layers = []
    current_width = width
    for i in range(depth):
        layers.append(int(current_width))
        if i < depth - 1:  # 最后一层不继续减半
            current_width = current_width / 2
    return layers

# base_model 的 resnet_layers 示例
base_resnet_layers = generate_resnet_layers(128, 6)
# e.g. [128, 64, 32, 16, 8, 4]




|   Exp |   d_model |   num_layers |   resnet_width |   resnet_depth |   learning_rate |   dropout_rate |
|------:|----------:|-------------:|---------------:|---------------:|----------------:|---------------:|
|     1 |        32 |            4 |             64 |              3 |          0.0001 |            0.1 |
|     2 |        32 |            8 |            128 |              6 |          0.001  |            0.3 |
|     3 |        32 |           12 |            256 |              9 |          0.01   |            0.5 |
|     4 |        64 |            4 |             64 |              6 |          0.001  |            0.5 |
|     5 |        64 |            8 |            128 |              9 |          0.01   |            0.1 |
|     6 |        64 |           12 |            256 |              3 |          0.0001 |            0.3 |
|     7 |       128 |            4 |            128 |              3 |          0.01   |            0.3 |
|     8 |       128 |            8 |            256 |              6 |          0.0001 |            0.5 |
|     9 |       128 |           12 |             64 |              9 |          0.001  |            0.1 |
|    10 |        32 |            4 |            256 |              9 |          0.001  |            0.3 |
|    11 |        32 |            8 |             64 |              3 |          0.01   |            0.5 |
|    12 |        32 |           12 |            128 |              6 |          0.0001 |            0.1 |
|    13 |        64 |            4 |            128 |              9 |          0.0001 |            0.5 |
|    14 |        64 |            8 |            256 |              3 |          0.001  |            0.1 |
|    15 |        64 |           12 |             64 |              6 |          0.01   |            0.3 |
|    16 |       128 |            4 |            256 |              6 |          0.001  |            0.1 |
|    17 |       128 |            8 |             64 |              9 |          0.01   |            0.3 |
|    18 |       128 |           12 |            128 |              3 |          0.0001 |            0.5 |


In [5]:
# 3. 正交实验设计（L18(3^6)）
def generate_l18_orthogonal_array():
    """
    生成L18(3^6)标准正交表
    18行（实验次数）× 6列（因子数）
    每列包含0,1,2三个水平的平衡分布
    """
    # 标准L18(3^6)正交表
    orthogonal_array = np.array([
        [0, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 1, 1],
        [0, 2, 2, 2, 2, 2],
        [1, 0, 0, 1, 1, 2],
        [1, 1, 1, 2, 2, 0],
        [1, 2, 2, 0, 0, 1],
        [2, 0, 1, 0, 2, 1],
        [2, 1, 2, 1, 0, 2],
        [2, 2, 0, 2, 1, 0],
        [0, 0, 2, 2, 1, 1],
        [0, 1, 0, 0, 2, 2],
        [0, 2, 1, 1, 0, 0],
        [1, 0, 1, 2, 0, 2],
        [1, 1, 2, 0, 1, 0],
        [1, 2, 0, 1, 2, 1],
        [2, 0, 2, 1, 1, 0],
        [2, 1, 0, 2, 2, 1],
        [2, 2, 1, 0, 0, 2]
    ])
    return orthogonal_array

# 生成正交表
orthogonal_array = generate_l18_orthogonal_array()
print("✅ L18(3^6)正交表生成完成")
print(f"  - 实验次数: {len(orthogonal_array)} (全因子设计需要 {3**6} 次)")
print(f"  - 减少实验次数: {3**6 - len(orthogonal_array)} 次 ({100*(1-len(orthogonal_array)/(3**6)):.1f}%)")
print(f"  - 正交表形状: {orthogonal_array.shape}")


✅ L18(3^6)正交表生成完成
  - 实验次数: 18 (全因子设计需要 729 次)
  - 减少实验次数: 711 次 (97.5%)
  - 正交表形状: (18, 6)


In [6]:
# 4. 参数映射函数（更新版：处理ResNet宽度和深度）
def map_orthogonal_to_params(orthogonal_array, param_levels):
    """
    将正交表中的水平索引映射到实际参数值
    特殊处理：将resnet_width和resnet_depth转换为resnet_layers列表
    
    参数:
    - orthogonal_array: 18×6的正交表数组
    - param_levels: 参数字典，每个参数对应3个水平值的列表
    
    返回:
    - param_combinations: 参数组合列表（包含转换后的resnet_layers）
    """
    param_names = list(param_levels.keys())
    param_combinations = []
    
    for row in orthogonal_array:
        param_dict = {}
        for i, param_name in enumerate(param_names):
            level_idx = row[i]
            param_dict[param_name] = param_levels[param_name][level_idx]
        
        # 特殊处理：将resnet_width和resnet_depth转换为resnet_layers
        if 'resnet_width' in param_dict and 'resnet_depth' in param_dict:
            resnet_layers = generate_resnet_layers(
                param_dict['resnet_width'], 
                param_dict['resnet_depth']
            )
            param_dict['resnet_layers'] = resnet_layers
            # 保留原始值用于记录
            param_dict['_resnet_width'] = param_dict.pop('resnet_width')
            param_dict['_resnet_depth'] = param_dict.pop('resnet_depth')
        
        # 确保num_heads与d_model兼容（d_model必须能被num_heads整除）
        if 'd_model' in param_dict:
            d_model = param_dict['d_model']
            # 根据d_model自动选择合适的num_heads
            if d_model == 32:
                param_dict['num_heads'] = 2  # 32/2=16
            elif d_model == 64:
                param_dict['num_heads'] = 4  # 64/4=16
            elif d_model == 96:
                param_dict['num_heads'] = 4  # 96/4=24（或8，但4更稳定）
        
        param_combinations.append(param_dict)
    
    return param_combinations

# 生成参数组合
param_combinations = map_orthogonal_to_params(orthogonal_array, sensitivity_configs)
print("✅ 参数组合映射完成")
print(f"  - 共 {len(param_combinations)} 个参数组合")
print(f"  - 示例组合 0:")
for k, v in param_combinations[0].items():
    print(f"      {k}: {v}")


✅ 参数组合映射完成
  - 共 18 个参数组合
  - 示例组合 0:
      d_model: 32
      num_layers: 4
      learning_rate: 0.0001
      dropout_rate: 0.1
      resnet_layers: [64, 32, 16]
      _resnet_width: 64
      _resnet_depth: 3
      num_heads: 2


## 3、Sensitivity analysis


核心目的：我训练生成了

模型当前的文件存储路径
```
 Supplymentary/
 └── Supplymentary/
     └── ML_sensitivity/
         └── results/
             ├── E1/
             └── ...
             └── E18/
```


 以下是存储文件的json结构（树形结构、以E2编号为例，一共有18个编号）
 
 我们主要关注的是d_model，num_layers，learning_rate，dropout_rate，resnet_layers，_resnet_width，_resnet_depth主要包含了这6个参数组成的水平（一共有18个）

```
 ├── d_model: 32
 ├── num_layers: 8
 ├── learning_rate: 0.001
 ├── dropout_rate: 0.3
 ├── resnet_layers: 
 │     └── [128, 64, 32, 16, 8, 4]
 ├── _resnet_width: 128
 ├── _resnet_depth: 6
 ├── num_heads: 2
 ├── overfitting_score: 0.01461287859752991
 ├── accuracy: 0.888474913065077
 ├── precision: 0.8754562043795621
 ├── recall: 0.9159904534606206
 ├── f1: 0.8952647539071612
 ├── mean_probability: '0.8077919'
 ├── exp_id: 'E2'
 ├── training_plot_path: 'results/E2/images/E2_training_results.png'
 ├── pipeline_plot_path: 'results/E2/images/E2_pipeline_analysis.png'
 ├── learning_curve_plot_path: 'results/E2/images/E2_learning_curve.png'
 ├── model_path: '/output/models/E2_transformer_generation.pkl'
 ├── uploaded_model_files: 
 │     ├── E2_transformer_generation_gmm.pkl
 │     ├── E2_transformer_generation_preprocessor.pkl
 │     ├── E2_transformer_generation_test_data.npz
 │     ├── E2_transformer_generation_config.json
 │     ├── E2_transformer_generation.pkl
 │     └── E2_transformer_generation_dl.h5
 └── model_gcs_path: 'gs://pv_cropland/results/E2/models/'
```


输出数据：包括实验的超参数调优、以及实验的主要精度结果
（1）首先是主要超参数结果dataframe

 仿照精度 dataframe，主要关注 6 个超参数：d_model, num_layers, learning_rate, dropout_rate, resnet_layers, resnet_width, _resnet_depth
 ```python
 Example:
 df_params = pd.DataFrame([{
     '实验编号': e2_results.get('exp_id', None),
     'd_model': e2_results.get('d_model', None),
     'num_layers': e2_results.get('num_layers', None),
     'learning_rate': e2_results.get('learning_rate', None),
     'dropout_rate': e2_results.get('dropout_rate', None),
     'resnet_layers': e2_results.get('resnet_layers', None),
     '_resnet_width': e2_results.get('_resnet_width', None),
     '_resnet_depth': e2_results.get('_resnet_depth', None)
 }])
 ```
（2）其次是主要精度结果dataframe
```python
df = pd.DataFrame([{
    '实验编号': e2_results.get('exp_id', None),
    'overfitting_score': e2_results.get('overfitting_score', None),
    'accuracy': e2_results.get('accuracy', None),
    'precision': e2_results.get('precision', None),
    'recall': e2_results.get('recall', None),
    'f1': e2_results.get('f1', None),
    'mean_probability': e2_results.get('mean_probability', None)
}])
```

 

In [3]:
# 按照相对目录结构打开文件 E2_results.json
from pathlib import Path
import json

# 构造 E2_results.json 的相对路径 (Supplymentary/ML_sensitivity/results/E2/E2_results.json)
results_file = Path('Supplymentary/ML_sensitivity/results/E2/E2_results.json')

# 读取文件内容
with open(results_file, 'r', encoding='utf-8') as f:
    e2_results = json.load(f)

e2_results

{'d_model': 32,
 'num_layers': 8,
 'learning_rate': 0.001,
 'dropout_rate': 0.3,
 'resnet_layers': [128, 64, 32, 16, 8, 4],
 '_resnet_width': 128,
 '_resnet_depth': 6,
 'num_heads': 2,
 'overfitting_score': 0.01461287859752991,
 'accuracy': 0.888474913065077,
 'precision': 0.8754562043795621,
 'recall': 0.9159904534606206,
 'f1': 0.8952647539071612,
 'mean_probability': '0.8077919',
 'exp_id': 'E2',
 'training_plot_path': 'results/E2/images/E2_training_results.png',
 'pipeline_plot_path': 'results/E2/images/E2_pipeline_analysis.png',
 'learning_curve_plot_path': 'results/E2/images/E2_learning_curve.png',
 'model_path': '/output/models/E2_transformer_generation.pkl',
 'uploaded_model_files': ['E2_transformer_generation_gmm.pkl',
  'E2_transformer_generation_preprocessor.pkl',
  'E2_transformer_generation_test_data.npz',
  'E2_transformer_generation_config.json',
  'E2_transformer_generation.pkl',
  'E2_transformer_generation_dl.h5'],
 'model_gcs_path': 'gs://pv_cropland/results/E2/model

In [5]:
# 读取所有实验的JSON文件并生成两个DataFrame
from pathlib import Path
import json
import pandas as pd

# 定义结果目录路径
results_base_dir = Path('Supplymentary/ML_sensitivity/results')

# 存储所有实验结果的列表
all_params_list = []
all_metrics_list = []

# 遍历E1到E18
for exp_num in range(1, 19):
    exp_id = f'E{exp_num}'
    json_file = results_base_dir / exp_id / f'{exp_id}_results.json'
    
    if json_file.exists():
        try:
            # 读取JSON文件
            with open(json_file, 'r', encoding='utf-8') as f:
                exp_results = json.load(f)
            
            # 提取超参数数据
            params_dict = {
                '实验编号': exp_results.get('exp_id', exp_id),
                'd_model': exp_results.get('d_model', None),
                'num_layers': exp_results.get('num_layers', None),
                'learning_rate': exp_results.get('learning_rate', None),
                'dropout_rate': exp_results.get('dropout_rate', None),
                'resnet_layers': exp_results.get('resnet_layers', None),
                '_resnet_width': exp_results.get('_resnet_width', None),
                '_resnet_depth': exp_results.get('_resnet_depth', None)
            }
            all_params_list.append(params_dict)
            
            # 提取精度结果数据
            metrics_dict = {
                '实验编号': exp_results.get('exp_id', exp_id),
                'overfitting_score': exp_results.get('overfitting_score', None),
                'accuracy': exp_results.get('accuracy', None),
                'precision': exp_results.get('precision', None),
                'recall': exp_results.get('recall', None),
                'f1': exp_results.get('f1', None),
                'mean_probability': exp_results.get('mean_probability', None)
            }
            all_metrics_list.append(metrics_dict)
            
        except Exception as e:
            print(f"⚠️ 读取 {exp_id} 时出错: {e}")
    else:
        print(f"⚠️ 文件不存在: {json_file}")

# 生成超参数DataFrame
df_params = pd.DataFrame(all_params_list)

# 生成精度结果DataFrame
df_metrics = pd.DataFrame(all_metrics_list)

# 按实验编号排序（确保E1到E18的顺序）
df_params = df_params.sort_values('实验编号').reset_index(drop=True)
df_metrics = df_metrics.sort_values('实验编号').reset_index(drop=True)



✅ 数据提取完成

超参数DataFrame (df_params):
  - 行数: 18
  - 列数: 8
  - 列名: ['实验编号', 'd_model', 'num_layers', 'learning_rate', 'dropout_rate', 'resnet_layers', '_resnet_width', '_resnet_depth']

精度结果DataFrame (df_metrics):
  - 行数: 18
  - 列数: 7
  - 列名: ['实验编号', 'overfitting_score', 'accuracy', 'precision', 'recall', 'f1', 'mean_probability']

超参数DataFrame预览:
   实验编号  d_model  num_layers  learning_rate  dropout_rate  \
0    E1       32           4         0.0001           0.1   
1   E10       32           4         0.0010           0.3   
2   E11       32           8         0.0100           0.5   
3   E12       32          12         0.0001           0.1   
4   E13       64           4         0.0001           0.5   
5   E14       64           8         0.0010           0.1   
6   E15       64          12         0.0100           0.3   
7   E16      128           4         0.0010           0.1   
8   E17      128           8         0.0100           0.3   
9   E18      128          12         0.00

In [ ]:
# 按照指定E1-E18顺序排序
exp_id_order = [f"E{i}" for i in range(1, 19)]
df_params['实验编号'] = pd.Categorical(df_params['实验编号'], categories=exp_id_order, ordered=True)
df_params = df_params.sort_values('实验编号').reset_index(drop=True)
df_params.to_csv(f"{results_base_dir}/hyparameters.csv", index=False)


In [ ]:
# 按照指定E1-E18顺序排序
exp_id_order = [f"E{i}" for i in range(1, 19)]
df_metrics['实验编号'] = pd.Categorical(df_metrics['实验编号'], categories=exp_id_order, ordered=True)
df_metrics = df_metrics.sort_values('实验编号').reset_index(drop=True)
df_metrics.to_csv(f"{results_base_dir}/metrics.csv", index=False)


In [12]:
df_metrics

,实验编号,overfitting_score,accuracy,precision,recall,f1,mean_probability
0,E1,9.509989e-03,0.844759,0.830783,0.881146,0.855224,0.7433181
1,E2,1.461288e-02,0.888475,0.875456,0.915990,0.895265,0.8077919
2,E3,1.022934e-02,0.876304,0.936577,0.817661,0.873089,0.71059215
3,E4,4.147772e-03,0.900397,0.893222,0.918377,0.905625,0.86266416
4,E5,-5.372834e-05,0.520368,0.520368,1.000000,0.684529,0.53029054
5,E6,8.213654e-03,0.854446,0.824516,0.915036,0.867421,0.77087706
6,E7,1.025066e-02,0.888227,0.876086,0.914558,0.894909,0.8704279
7,E8,-2.784090e-03,0.751366,0.791889,0.708353,0.747795,0.6862237
8,E9,-6.214761e-04,0.520368,0.520368,1.000000,0.684529,0.53161913
9,E10,4.505275e-03,0.887978,0.900585,0.882100,0.891247,0.8309969


In [14]:
from pathlib import Path
import json
import pandas as pd
import glob

# 定义模型目录路径
model_base_dir = DATA_PATH / 'US_data/ML_model'

# 存储所有模型结果的列表
all_params_list = []
all_metrics_list = []

# 查找所有 *_config.json 文件
config_files = glob.glob(str(model_base_dir / '*_config.json'))

print(f"找到 {len(config_files)} 个模型配置文件")

for config_file in sorted(config_files):
    try:
        # 读取配置文件
        with open(config_file, 'r', encoding='utf-8') as f:
            config_data = json.load(f)
        
        # 从文件名提取模型信息
        file_name = Path(config_file).stem
        # 格式: landuse_transformer_generation_single_20251124_234807_config
        parts = file_name.replace('_config', '').split('_')
        model_type = parts[1] if len(parts) > 1 else 'unknown'
        timestamp = parts[-1] if len(parts) > 1 else 'unknown'
        exp_id = f"{model_type}_{timestamp}"
        
        # 提取超参数
        model_config = config_data.get('model_config', {})
        model_params = model_config.get('model_params', {})
        
        # 提取 transformer 相关参数（如果存在）
        transformer_config = model_config.get('transformer_config', {})
        resnet_layers = model_config.get('resnet_layers', model_params.get('resnet_layers'))
        
        # 从 dl_architecture 推断参数（如果可能）
        dl_arch = config_data.get('dl_architecture', {})
        d_model = None
        num_layers = None
        num_heads = None
        
        # 尝试从 layers 中推断 transformer 参数
        if dl_arch and 'layers' in dl_arch:
            layers = dl_arch['layers']
            # 查找 transformer 相关层
            for layer in layers:
                if isinstance(layer, dict):
                    layer_name = layer.get('name', '')
                    if 'dense' in layer_name.lower() and 'units' in layer:
                        # 第一个 dense 层的 units 可能是 d_model
                        if d_model is None and layer.get('units') in [32, 64, 128]:
                            d_model = layer.get('units')
                    if 'multi_head_attention' in layer_name.lower() or 'attention' in layer_name.lower():
                        if 'num_heads' in layer:
                            num_heads = layer.get('num_heads')
        
        # 如果没有找到，使用默认值或从 transformer_config 获取
        if transformer_config:
            d_model = transformer_config.get('d_model', d_model)
            num_layers = transformer_config.get('num_layers', num_layers)
            num_heads = transformer_config.get('num_heads', num_heads)
        
        # 提取 ResNet 参数
        if isinstance(resnet_layers, list) and len(resnet_layers) > 0:
            _resnet_width = resnet_layers[0] if resnet_layers else None
            _resnet_depth = len(resnet_layers)
        else:
            _resnet_width = None
            _resnet_depth = None
        
        # 构建超参数字典
        params_dict = {
            '实验编号': exp_id,
            'model_type': model_type,
            'd_model': d_model,
            'num_layers': num_layers,
            'num_heads': num_heads,
            'learning_rate': model_params.get('learning_rate'),
            'dropout_rate': model_params.get('dropout_rate'),
            'resnet_layers': resnet_layers,
            '_resnet_width': _resnet_width,
            '_resnet_depth': _resnet_depth,
            'epochs': model_params.get('epochs'),
            'batch_size': model_params.get('batch_size'),
            'timestamp': timestamp
        }
        all_params_list.append(params_dict)
        
        # 提取精度结果数据
        training_metrics = config_data.get('training_metrics', {})
        test_metrics = training_metrics.get('test', {})
        
        # 计算 overfitting_score（如果有学习曲线分析）
        overfitting_score = None
        learning_curve_analysis = config_data.get('learning_curve_analysis', {})
        if learning_curve_analysis:
            overfitting_analysis = learning_curve_analysis.get('overfitting_analysis', {})
            overfitting_score = overfitting_analysis.get('final_gap')
        
        # 计算 mean_probability（从 prediction_results 或 final_results）
        mean_probability = None
        # 注意：mean_probability 可能不在 config.json 中，需要从其他文件读取
        
        metrics_dict = {
            '实验编号': exp_id,
            'model_type': model_type,
            'overfitting_score': overfitting_score,
            'accuracy': test_metrics.get('accuracy'),
            'precision': test_metrics.get('precision'),
            'recall': test_metrics.get('recall'),
            'f1': test_metrics.get('f1'),
            'auc': test_metrics.get('auc'),
            'mean_probability': mean_probability,
            'timestamp': timestamp
        }
        all_metrics_list.append(metrics_dict)
        
    except Exception as e:
        print(f"⚠️ 读取 {config_file} 时出错: {e}")
        import traceback
        traceback.print_exc()

# 生成超参数DataFrame
df_params = pd.DataFrame(all_params_list)

# 生成精度结果DataFrame
df_metrics = pd.DataFrame(all_metrics_list)

# 按实验编号排序
if not df_params.empty:
    df_params = df_params.sort_values('实验编号').reset_index(drop=True)
if not df_metrics.empty:
    df_metrics = df_metrics.sort_values('实验编号').reset_index(drop=True)

print("=" * 80)
print("✅ 数据提取完成")
print("=" * 80)
print(f"\n超参数DataFrame (df_params):")
print(f"  - 行数: {len(df_params)}")
if not df_params.empty:
    print(f"  - 列数: {len(df_params.columns)}")
    print(f"  - 列名: {list(df_params.columns)}")

print(f"\n精度结果DataFrame (df_metrics):")
print(f"  - 行数: {len(df_metrics)}")
if not df_metrics.empty:
    print(f"  - 列数: {len(df_metrics.columns)}")
    print(f"  - 列名: {list(df_metrics.columns)}")

print("\n" + "=" * 80)
print("超参数DataFrame预览:")
print("=" * 80)
if not df_params.empty:
    print(df_params)
else:
    print("无数据")

print("\n" + "=" * 80)
print("精度结果DataFrame预览:")
print("=" * 80)
if not df_metrics.empty:
    print(df_metrics)
else:
    print("无数据")

找到 4 个模型配置文件
✅ 数据提取完成

超参数DataFrame (df_params):
  - 行数: 4
  - 列数: 13
  - 列名: ['实验编号', 'model_type', 'd_model', 'num_layers', 'num_heads', 'learning_rate', 'dropout_rate', 'resnet_layers', '_resnet_width', '_resnet_depth', 'epochs', 'batch_size', 'timestamp']

精度结果DataFrame (df_metrics):
  - 行数: 4
  - 列数: 10
  - 列名: ['实验编号', 'model_type', 'overfitting_score', 'accuracy', 'precision', 'recall', 'f1', 'auc', 'mean_probability', 'timestamp']

超参数DataFrame预览:
                 实验编号   model_type  d_model num_layers num_heads  \
0          mlp_144212          mlp    128.0       None      None   
1           rf_212051           rf      NaN       None      None   
2  transformer_212041  transformer     64.0       None      None   
3  transformer_234807  transformer     64.0       None      None   

   learning_rate  dropout_rate resnet_layers _resnet_width _resnet_depth  \
0          0.001           0.3          None          None          None   
1          0.001           0.3          None   

In [20]:
import pandas as pd 

df_main_dir = DATA_PATH / 'US_data/ML_model/results_transformer_20251124_234807.csv'
df_main = pd.read_csv(df_main_dir)
df_main_notnull = df_main[df_main['lat'].notna() & df_main['lon'].notna()]
df_main_notnull['predicted_prob'].mean()

0.8403101414085618